**OBJETIVO DEL PROYECTO**

Construir un sistema de recomendación de películas basado en contenido textual, utilizando las descripciones (overview) de las películas para encontrar similitudes y sugerir títulos parecidos.



*   Para convertir los textos (por ejemplo, el overview) en vectores numéricos se utilizará: TF-IDF (Term Frequency – Inverse Document Frequency)
*   Para medir qué tan similares son dos películas basadas en sus vectores de característica se utilizará: Cosine Similarity





In [26]:
import pandas as pd
df = pd.read_csv("/content/tmdb_5000_movies.csv")
df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [27]:
print(df.shape)
print(df.info())

(4803, 20)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status    

**Nota:**


1.   A diferencia de modelos tradicionales no es necesario graficar la variable de interés (overview), ya que, no se puede graficar una distribución de texto como lo harías con una variable numérica (por ejemplo, con un histograma. Tampoco tiene sentido calcular su correlación con otras variables, porque no tiene valor cuantitativo directo
2.   A su vez, no tiene sentido calcular asimetría o curtosis de esta variable ya que, estas son medidas de distribución estadística aplicables a variables numéricas continuas, como ingresos, precios, edades, etc.
3.   Por último, no es necesario aplicar normalizado y balance de clases debido a que, usamos TF-IDF, que ya entrega vectores normalizados internamente y luego calculamos cosine similarity, que ya es una medida basada en dirección (ángulo, hacia donde), no magnitud, por lo tanto no requiere escalamiento extra






Elegimos solo las columnas necesarias

In [28]:
df.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

**Por ahora trabajaremos con**: title y overview

Revisar valores nulos


In [29]:

df[['title', 'overview']].isnull().sum()

,0
title,0
overview,3


In [30]:
df1= df.dropna(subset=['overview'])

In [31]:
df1[["overview"]].isnull().sum()

,0
overview,0


También eliminamos duplicados si existen:

In [32]:
df1 = df.drop_duplicates(subset='title')

 **Preprocesamiento de texto**

El campo overview contiene texto, así que tenemos que convertirlo en números (vectores). Usamos: TF-IDF.

In [37]:
df1['overview'] = df1['overview'].fillna('')  #Por si acaso, se aplica.

/tmp/ipython-input-37-1019756596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['overview'] = df1['overview'].fillna('')


In [38]:
df1['overview'] = df1['overview'].astype(str) #Por si acaso, se aplica.

/tmp/ipython-input-38-659349236.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['overview'] = df1['overview'].astype(str)


In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')




*   TfidfVectorizer: Devuelve una matriz numérica (vectores)
*   Stop words: Son palabras comunes que aparecen con mucha frecuencia en un idioma pero que no aportan mucho significado (the, is , a), se utiliza para filtrar. Cuando aplicamos el vectorizador, este eliminará automáticamente esas palabras.





In [40]:
tfidf_matrix = tfidf.fit_transform(df1['overview'])





*  fit(): Aprende el vocabulario de todas las sinopsis y calcula los pesos IDF.
*   transform(): Convierte cada sinopsis en un vector numérico basado en TF-IDF.



In [41]:
tfidf_matrix

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 125704 stored elements and shape (4800, 20967)>

Tenemos una matriz de:

4800 filas: Número de documentos/películas.

20967 columnas: Palabras únicas en el vocabulario TF-IDF.

**Podemos podemos medir qué tan similares son entre sí.**

Usamos una medida llamada cosine similarity (similitud del coseno).

Como tenemos vectores (cada película) en un espacio. El coseno del ángulo entre dos vectores mide su similitud.

**A continuación tendremos:**

cosine_similarity: Es una métrica que mide qué tan similares son dos vectores en un espacio multidimensional, basándose en el ángulo entre ellos.

(tfidf_matrix, tfidf_matrix): Ambas matrices representan el mismo conjunto de películas, y al pasarlas dos veces, estamos pidiendo la similitud entre todas las películas entre sí.

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [43]:
print(cosine_sim.shape)  #(n_películas x n_películas)

(4800, 4800)


**cosine_sim[i][j]**: te da el valor de similitud entre la película con índice i y la película con índice j.

Creamos una estructura que nos permite buscar rápidamente el índice numérico de cualquier película usando su título como clave

In [44]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [45]:
indices

,0
title,
Avatar,0
Pirates of the Caribbean: At World's End,1
Spectre,2
The Dark Knight Rises,3
John Carter,4
...,...
El Mariachi,4798
Newlyweds,4799
"Signed, Sealed, Delivered",4800


In [52]:
def recomendar_peliculas(titulo, n=5): #La función toma un título de película y un número n de recomendaciones a devolver (por defecto 5).

    idx = indices[titulo] #Usa la Serie creada anteriormente para encontrar el índice numérico de la película con ese título (el título que ingresamos en la función). Es decir, ingresa a la serie con el nombre del título y me da un número de índice.


    sim_scores = list(enumerate(cosine_sim[idx])) #cosine_sim[idx] extrae la fila idx de la matriz cosine_sim, que representa la similitud de esa película con todas las demás. Y me da un array donde cada número es su similitud con cada película. Enumerate, agrega un número a cada puntuación. List convierte la tuplas que creo (1, 0.8) en una lista (sigue teniendo valores:(1, 0.8)) para poder recorrerla, etc.


    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #Ordenar esas tuplas (obtenidas anteriormente en formato lista) por su segundo elemento (el valor de similitud), de mayor a menor, para que las películas más parecidas queden al principio de la lista.


    sim_scores = sim_scores[1:n+1] #usamos slicing en lista para que empieza en el índice 1, o sea, saltea el primero (es la película que buscamos) y n+1 → para que incluya el índice n. Obtenemos una lista con el top n que elegimos.


    movie_indices = [i[0] for i in sim_scores] #Aplicamos lista por comprensión, para obtener el índice del top N de películas.


    return df[['title', 'overview']].iloc[movie_indices] #El retorno de la función es el título y sinopsis de cada película, usamos iloc para acceder a filas por número de índice (no por nombre de columna o ID).

**Probar la función**

Para probar el funcionamiento del sistema, se utilizó la película "The Dark Knight" como punto de partida

In [53]:
recomendar_peliculas("The Dark Knight", n=5)

,title,overview
3,The Dark Knight Rises,Following the death of District Attorney Harve...
428,Batman Returns,"Having defeated the Joker, Batman now faces th..."
3852,The Secret,The Secret is the story of a real-life double ...
299,Batman Forever,The Dark Knight of Gotham City confronts a das...
1359,Batman,The Dark Knight of Gotham City begins his war ...


Como vemos anteriormente, el sistema devolvió las 5 películas más similares en función de su sinopsis (overview), basándose exclusivamente en la similitud textual. Esto demuestra que el modelo es capaz de encontrar contenido relevante en función del texto, incluso reconociendo relaciones temáticas y de saga entre películas.

A su vez, **el modelo es adaptable a distintas columnas**, el sistema de recomendación está construido de forma flexible, permitiendo que se adapte fácilmente a diferentes criterios de similitud, simplemente modificando la columna utilizada como base del análisis textual.

En su versión inicial, el modelo utiliza la columna overview (sinopsis de las películas) para calcular la similitud entre películas mediante la combinación de TF-IDF (Term Frequency-Inverse Document Frequency) y cosine similarity. Sin embargo, si se desea recomendar películas en función de otro aspecto —como genres, keywords, director, etc.— basta con reemplazar la columna de entrada.  Requisito: La columna utilizada debe estar en formato de texto plano (string)